In [ ]:
!pip install datasets spacy pandas


In [ ]:
!pip install better_profanity

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 2.5 MB/s eta 0:00:00


In [ ]:
import spacy
import pandas as pd
import random
from datasets import load_dataset

In [ ]:
ds = load_dataset("nilq/babylm-10M")
print(ds)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/883 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/36.2M [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/34.6M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/37.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1058740 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1026747 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1054646 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 1058740
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 1026747
    })
    test: Dataset({
        features: ['text'],
        num_rows: 1054646
    })
})


In [ ]:
dataset = ds if not isinstance(ds, dict) else ds[list(ds.keys())[0]]
print("Number of examples:", len(dataset))

Number of examples: 1058740


In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
def extract_key_word_doc(doc):
    # Prefer NOUN / PROPN first
    for token in doc:
        if token.pos_ in ["NOUN", "PROPN"]:
            return token.text.lower()
    # Fallback to VERB
    for token in doc:
        if token.pos_ in ["VERB"]:
            return token.text.lower()
    return None

In [ ]:
# Define Subset
subset_size = 650000

subset = dataset.select(range(subset_size))

# extract sentences
sentences = [ex['text'].strip() for ex in subset if len(ex['text'].strip().split()) >= 2]

print(f"Processing {len(sentences)} sentences...")

Processing 568580 sentences...


In [ ]:
# Profanity Filter
from better_profanity import profanity
profanity.load_censor_words()
sentences = [s for s in sentences if not profanity.contains_profanity(s)]
print(f"After profanity removal: {len(sentences)} sentences remain.")

After profanity removal: 552734 sentences remain.


In [ ]:
def is_reasonable_length(doc, min_len=2, max_len=12):
    return min_len <= len(doc) <= max_len

ADULT_POS = {"ADV", "ADJ"}
PAST_VERBS = {"VBD", "VBN"}

def has_adult_markers(doc):
    if sum(1 for t in doc if t.pos_ in ADULT_POS) >= 3:
        return True
    # contains past tense narration
    if any(t.tag_ in PAST_VERBS for t in doc):
        return True
    return False

def has_cdi_word(doc, CDI_set):
    return any(token.text.lower() in CDI_set for token in doc)

def is_child_directed(doc, CDI_set):
    if not is_reasonable_length(doc):
        return False
    if has_adult_markers(doc):
        return False
    if not has_cdi_word(doc, CDI_set):
        return False
    return True


In [ ]:
pairs = []
CDI_words=['light', 'dog', 'hi', 'home', 'kitchen', 'milk', 'girl', 'break', 'tooth', 'flower', 'bed', 'soft', 'duck', 'tv', 'block', 'ouch', 'kiss', 'blanket', 'dish', 'meow', 'ball', 'fast', 'jump', 'spoon', 'candy', 'please', 'sock', 'eye', 'bear', 'sun', 'boat', 'fish', 'feed', 'pants', 'kick', 'grandma', 'cookie', 'night', 'pattycake', 'moon', 'box', 'bread', 'chair', 'rock', 'mouth', 'hand', 'apple', 'clock', "don't", 'cat', 'head', 'out', 'phone', 'eyes', 'away', 'mommy', 'lamp', 'how', 'leg', 'bird', 'smile', 'broken', 'lion', 'banana', 'toe', 'nose', 'other', 'plant', 'help', 'push', 'bottle', 'outside', 'dark', 'sing', 'cow', 'truck', 'kitty', 'bus', 'radio', 'rain', 'daddy', 'cereal', 'shoe', 'couch', 'toast', 'mouse', 'juice', 'pretty', 'hurt', 'finish', 'star', 'car', 'me', 'today', 'keys', 'book', 'doll', 'bath', 'who', 'baby', 'wait', 'hair', 'stroller', 'table', 'ear', 'hat', 'some', 'foot', 'cup', 'coat', 'big', 'i', 'babysitter']
for doc, sentence in zip(nlp.pipe(sentences, batch_size=1000), sentences):
    # reject adult-like sentences
    if not is_child_directed(doc, CDI_words):
        continue

    key_word = extract_key_word_doc(doc)
    if key_word:
        pairs.append({"adult": sentence.strip(), "child": key_word})

random.shuffle(pairs)
print(f"Generated {len(pairs)} synthetic MOT→CHI pairs (CDS-filtered).")


Generated 78528 synthetic MOT→CHI pairs (CDS-filtered).


In [ ]:
df = pd.DataFrame(pairs)

def nlp_tie_breaker(child_words, adult_sentence, CDI_words, nlp):
    if isinstance(child_words, str):
        child_words = child_words.lower().split()

    # Step 1: Filter CDI matches
    matches = [w for w in child_words if w in CDI_words]
    if not matches:
        return None

    # Parse adult sentence
    doc = nlp(str(adult_sentence).lower())

    # Step 2: Compute correctness score
    scores = {m: 0 for m in matches}
    for token in doc:
        lemma = token.lemma_.lower()
        text = token.text.lower()
        for m in matches:
            if text == m or lemma == m:
                if token.pos_ in {"NOUN", "PROPN"}:
                    scores[m] += 1
                if token.dep_ in {"dobj", "pobj", "attr"}:
                    scores[m] += 2

    # Step 3: Pick the highest-scoring CDI word
    best_match = max(scores, key=scores.get)
    best_score = scores[best_match]
    return best_match, best_score

child_words = set(df["child"].astype(str).unique()) - {"unknown"}

# Note : No unkown were generated (0) as a part of this pipeline
def generate_unknown_pairs(df, CDI_words, child_words, max_unknowns=5000):
    unknown_pairs = []
    for _, row in df.iterrows():
        adult_sentence = str(row["adult"])
        adult_l = adult_sentence.lower()
        doc = nlp(adult_l)

        tokens = [t.text.lower() for t in doc if t.is_alpha]

        # Skip if any token is a known child word
        if any(tok in child_words for tok in tokens):
            continue

        candidates = [tok for tok in tokens if tok not in CDI_words]

        if candidates:
            chosen = random.choice(candidates)
            unknown_pairs.append({
                "adult": row["adult"],
                "child": "unknown",
                "unknown_token": chosen,
            })

        if len(unknown_pairs) >= max_unknowns:
            break

    return pd.DataFrame(unknown_pairs)

In [ ]:

df[["child_filtered", "correctness_score"]] = df.apply(
    lambda row: pd.Series(nlp_tie_breaker(row["child"], row["adult"], CDI_words, nlp)),
    axis=1
)

filtered_df = df[df["child_filtered"].notnull()]

num_unknowns = min(5000, len(filtered_df)//10)
unknown_df = generate_unknown_pairs(filtered_df, CDI_words,child_words,max_unknowns=num_unknowns)

final_df = pd.concat([filtered_df, unknown_df], ignore_index=True)


print(len(filtered_df))
print(len(unknown_df))
print(len(final_df))
print("Sample synthetic dataset:")
print(final_df.head(20))
output_path = "synthetic_CDI_pairs.csv"
final_df.to_csv(output_path, index=False)
print(f"\n Filtered CDI pairs saved to: {output_path}")